<a href="https://colab.research.google.com/github/meggilliam/datasets/blob/main/Project1_Group4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Source: [ICS.UCI.EDU](https://archive.ics.uci.edu/ml/datasets/Wine+Quality)

In [4]:
# Setup the Environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [ ]:
# Import the Dataset for Red Wine - hosted on Github
url = 'https://raw.githubusercontent.com/meggilliam/datasets/main/winequality-red.csv'
wine = pd.read_csv(url)

print(wine.head)

In [5]:
# Run SVM

In [6]:
# Run NN

In [7]:
# Run Third Algorithm